In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# turning of warnings
import warnings
warnings.filterwarnings("ignore")
import pprint
import os

# changing working directory to repositories
os.chdir('/content/drive/MyDrive/repositories/Generative-AI')

Mounted at /content/drive


In [ ]:
# %pip install transformers torch
import torch
import transformers
print('torch version:', torch.__version__)
print('transformers version:', transformers.__version__)

torch version: 2.8.0+cu126
transformers version: 4.56.1


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: ',device)

device:  cpu


# Tokenizers

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [ ]:
pprint.pprint(tokenizer)

BartTokenizerFast(name_or_path='sshleifer/distilbart-cnn-12-6', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}
)


In [ ]:
raw_inputs =     [
    "I love deep learning  & machine learning!",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
pprint.pprint(inputs)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]),
 'input_ids': tensor([[   0,  100,  657, 1844, 2239, 1437,  359, 3563, 2239,  328,    2],
        [   0,  100, 4157,   42,   98,  203,  328,    2,    1,    1,    1]])}


In [ ]:
inputs['input_ids'].shape

torch.Size([2, 11])

In [ ]:
print(raw_inputs[0])
print(f"Input ids: {inputs['input_ids'][0]}")
print(f"Attention Mask: {inputs['attention_mask'][0]}")
print("-"*100)
print(raw_inputs[1])
print(f"Input ids: {inputs['input_ids'][1]}")
print(f"Attention Mask: {inputs['attention_mask'][1]}")

I love deep learning  & machine learning!
Input ids: tensor([   0,  100,  657, 1844, 2239, 1437,  359, 3563, 2239,  328,    2])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
----------------------------------------------------------------------------------------------------
I hate this so much!
Input ids: tensor([   0,  100, 4157,   42,   98,  203,  328,    2,    1,    1,    1])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])


In [ ]:
tokenizer('I love deep learning & machine learning')

{'input_ids': [0, 100, 657, 1844, 2239, 359, 3563, 2239, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokens = tokenizer.tokenize('I love deep learning & machine learning')
tokens

['I', 'Ġlove', 'Ġdeep', 'Ġlearning', 'Ġ&', 'Ġmachine', 'Ġlearning']

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[100, 657, 1844, 2239, 359, 3563, 2239]

In [ ]:
decoded_tokens = tokenizer.decode(token_ids)
decoded_tokens

'I love deep learning & machine learning'

In [ ]:
model_prepped_ids = tokenizer.prepare_for_model(token_ids)
model_prepped_ids

{'input_ids': [0, 100, 657, 1844, 2239, 359, 3563, 2239, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
model_prepped_ids = tokenizer.prepare_for_model(token_ids)
model_prepped_ids

{'input_ids': [0, 100, 657, 1844, 2239, 359, 3563, 2239, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Pipeline

In [ ]:
from transformers import pipeline

classifier = pipeline(
    task = "sentiment-analysis",
    model='distilbert/distilbert-base-uncased-finetuned-sst-2-english',
    device='cpu',
)
classification_outputs = classifier(
    [
        "I love deep learning!",
        "I hate this so much!",
    ]
)

Device set to use cpu


In [ ]:
classification_outputs

[{'label': 'POSITIVE', 'score': 0.9998645782470703},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
text_generator = pipeline("text-generation",model="openai-community/gpt2")

text_generator_outputs = text_generator([
    'I went to the store to buy',
    'When two objects in space get close to each other'
])

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
text_generator_outputs

[[{'generated_text': "I went to the store to buy my new Honda. I was scared of a black Honda. My Honda was on the back of a big box in the back, so I went in and bought a new one. I had to buy new wheels, I had to buy new tires, and I had to buy new tires. It was all very expensive, and it was all very intimidating. I was going to buy a new car. And it took me a while to get used to it. I went out and bought a new car. And it took me a while to get used to. I was going to buy a new car. And it took me a while to get used to it. My new Honda was so good.\n\nI had my new Honda in my trunk. The only thing about this car that made me feel like I could go out and buy a new one was the fact that I could take it home. It was like getting a new car when you get older. You don't want to go to the store and have to buy a new car to drive. You want to have a car that you can go out and drive with. That was my new Honda. I had my new Honda in my trunk. The only thing about this car that made me fe

In [ ]:
summarizer = pipeline("summarization",model="sshleifer/distilbart-cnn-12-6")

generated_summary = summarizer([
    """A Fibonacci heap is a collection of trees satisfying the min-heap property. It allows faster amortized time for many operations than binary or binomial heaps.
    Trees in a Fibonacci heap can have any shape, which facilitates efficient operations. Lazy strategies are employed: node removals and consolidations are delayed until
    absolutely necessary (like during an extract-min operation). The main advantage lies in decreasing a key and merging two heaps, which are constant and amortized
    constant time, respectively. Nodes have a "mark" indicating if they've lost a child since the last time they were made a child of another node, assisting in
    restructuring during operations."""
])

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
generated_summary

[{'summary_text': ' A Fibonacci heap is a collection of trees satisfying the min-heap property . It allows faster amortized time for many operations than binary or binomial heaps . Nodes have a "mark" indicating if they\'ve lost a child since the last time they were made a child of another node .'}]

# Accessing Pretrained models

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
inputs = tokenizer('I love deep learning', return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 1045, 2293, 2784, 4083,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.1975,  4.4937]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Model Embeddings

In [ ]:
from transformers import AutoModel
model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
inputs = tokenizer('I love deep learning!', padding=True, truncation=True, return_tensors='pt')
print(inputs)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape) # the token embeddings

{'input_ids': tensor([[ 101, 1045, 2293, 2784, 4083,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
torch.Size([1, 7, 768])


In [ ]:
outputs

BaseModelOutput(last_hidden_state=tensor([[[ 0.6121,  0.0388,  0.0428,  ...,  0.4086,  0.9149, -0.3610],
         [ 0.8534,  0.1787, -0.0262,  ...,  0.3572,  0.9324, -0.2028],
         [ 0.8895,  0.2892,  0.1768,  ...,  0.3358,  0.8173, -0.1662],
         ...,
         [ 0.2977,  0.0795,  0.0422,  ...,  0.3568,  0.9583, -0.1966],
         [ 0.7450,  0.1104,  0.0711,  ...,  0.4781,  0.8737, -0.2799],
         [ 1.2192,  0.0954,  0.4204,  ...,  0.6585,  0.5959, -0.7534]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [ ]:
# to get the full context vector for the sequence
context_vectors = outputs.last_hidden_state.mean(dim=1)
context_vectors.shape

torch.Size([1, 768])

# Accessing Model Config & Creating Custom Models

In [ ]:
from transformers import GPT2Config, GPT2Model

# Building the config
config = GPT2Config()

In [ ]:
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.56.1",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
# Building the model from the config
gpt_model = GPT2Model(config)

# Saving Models

In [ ]:
# gpt_model.save_pretrained('gpt2_model')